In [1]:
import pandas as pd

In [2]:
# Read in the data
house_df = pd.read_csv('assets/updated_house_price.csv')
pop_growth_df_1 = pd.read_csv('assets/population_growth_2000_2010.csv', encoding='latin-1')
pop_growth_df_2 = pd.read_csv('assets\population_growth_2010-2020.csv', encoding='latin-1')
pop_growth_df_3 = pd.read_csv('assets/population_growth_2020-2021.csv', encoding='latin-1')
unemploy_df = pd.read_csv('assets/wake_unemployment.csv')
covid_df = pd.read_csv('assets/covid_confirmed.csv')
mort_rate_df = pd.read_csv('assets/mort_rate.csv')
lumber_df = pd.read_csv('assets/lumber.csv')
metals_df = pd.read_csv('assets/metals.csv')
build_df = pd.read_csv('assets/build_material.csv')

## Mortgage rate

In [3]:
# Work on the mortgage rate data
mort_rate_df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
mort_rate_df['date'] = pd.to_datetime(mort_rate_df['date'])
mort_rate_df['date'] = mort_rate_df['date'].dt.strftime('%Y-%m')
mort_rate_df.head()

,date,Rate
0,2000-01,8.21
1,2000-02,8.33
2,2000-03,8.24
3,2000-04,8.15
4,2000-05,8.52


In [4]:
house_df['Deed_Date'] = pd.to_datetime(house_df['Deed_Date'])
house_df['Deed_Date'] = house_df['Deed_Date'].dt.strftime('%Y-%m')
house_df.drop(columns=['Unnamed: 0'], inplace=True)
house_df

,Land_Sale_Price,Total_sale_Price,Deed_Date,Assessed_Building_Value,Story_Height,HEATED_AREA,UTILITIES,BATH,TYPE_AND_USE,PHYSICAL_ZIP_CODE,PHYSICAL_CITY
0,0.0,37500.0,2004-09,118723.0,1.00,2261,WSE,2.0,1.0,27591,WENDELL
1,0.0,380000.0,2015-08,161077.0,1.00,1789,ALL,2.0,1.0,27607,RALEIGH
2,0.0,337500.0,2012-12,273621.0,1.00,2463,ALL,2.0,1.0,27615,RALEIGH
3,0.0,319000.0,2010-06,503301.0,1.50,4650,ALL,3.5,1.0,27608,RALEIGH
4,0.0,425000.0,2008-04,203178.0,2.00,1890,ALL,2.0,1.0,27604,RALEIGH
...,...,...,...,...,...,...,...,...,...,...,...
286828,0.0,1194000.0,2022-08,547787.0,1.88,4520,ALL,3.5,1.0,27502,APEX
286829,0.0,1364500.0,2022-07,560914.0,1.88,4347,ALL,3.5,1.0,27502,APEX
286830,0.0,220000.0,2022-06,93432.0,1.00,896,ALL,1.0,1.0,27529,GARNER
286831,0.0,170000.0,2021-05,119201.0,1.00,896,ALL,1.0,1.0,27529,GARNER


### Combine mortgage rate to house price data

In [5]:
updated_house_df = house_df.merge(mort_rate_df, left_on='Deed_Date', right_on='date', how='left')
updated_house_df.rename(columns={'Rate': 'mort_rate'}, inplace=True)
updated_house_df.drop(columns=['date'], inplace=True)
updated_house_df.columns = map(str.lower, updated_house_df.columns)
updated_house_df

,land_sale_price,total_sale_price,deed_date,assessed_building_value,story_height,heated_area,utilities,bath,type_and_use,physical_zip_code,physical_city,mort_rate
0,0.0,37500.0,2004-09,118723.0,1.00,2261,WSE,2.0,1.0,27591,WENDELL,5.75
1,0.0,380000.0,2015-08,161077.0,1.00,1789,ALL,2.0,1.0,27607,RALEIGH,3.91
2,0.0,337500.0,2012-12,273621.0,1.00,2463,ALL,2.0,1.0,27615,RALEIGH,3.35
3,0.0,319000.0,2010-06,503301.0,1.50,4650,ALL,3.5,1.0,27608,RALEIGH,4.74
4,0.0,425000.0,2008-04,203178.0,2.00,1890,ALL,2.0,1.0,27604,RALEIGH,5.92
...,...,...,...,...,...,...,...,...,...,...,...,...
286828,0.0,1194000.0,2022-08,547787.0,1.88,4520,ALL,3.5,1.0,27502,APEX,5.22
286829,0.0,1364500.0,2022-07,560914.0,1.88,4347,ALL,3.5,1.0,27502,APEX,5.41
286830,0.0,220000.0,2022-06,93432.0,1.00,896,ALL,1.0,1.0,27529,GARNER,5.52
286831,0.0,170000.0,2021-05,119201.0,1.00,896,ALL,1.0,1.0,27529,GARNER,2.96


## Unemployment rate

In [6]:
unemploy_df['Year'] = unemploy_df['Year'].astype(str)
unemploy_df['Month'] = unemploy_df['Month'].astype(str)
unemploy_df['date'] = unemploy_df['Year'] + '-' + unemploy_df['Month']
unemploy_df['date'] = pd.to_datetime(unemploy_df['date'])
unemploy_df['date'] = unemploy_df['date'].dt.strftime('%Y-%m')
unemploy_df.rename(columns={'Unemployment Rate(%)': 'unemploy_rate'}, inplace=True)

### Combine unemployment rate to house price data

In [7]:
unemploy_df = unemploy_df[['date', 'unemploy_rate']].copy()
updated_house_df = updated_house_df.merge(unemploy_df, left_on='deed_date', right_on='date', how='left')
updated_house_df.drop(columns=['date'], inplace=True)
# updated_house_df.columns = map(str.lower, updated_house_df.columns)
updated_house_df

,land_sale_price,total_sale_price,deed_date,assessed_building_value,story_height,heated_area,utilities,bath,type_and_use,physical_zip_code,physical_city,mort_rate,unemploy_rate
0,0.0,37500.0,2004-09,118723.0,1.00,2261,WSE,2.0,1.0,27591,WENDELL,5.75,4.2
1,0.0,380000.0,2015-08,161077.0,1.00,1789,ALL,2.0,1.0,27607,RALEIGH,3.91,4.9
2,0.0,337500.0,2012-12,273621.0,1.00,2463,ALL,2.0,1.0,27615,RALEIGH,3.35,7.0
3,0.0,319000.0,2010-06,503301.0,1.50,4650,ALL,3.5,1.0,27608,RALEIGH,4.74,8.4
4,0.0,425000.0,2008-04,203178.0,2.00,1890,ALL,2.0,1.0,27604,RALEIGH,5.92,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
286828,0.0,1194000.0,2022-08,547787.0,1.88,4520,ALL,3.5,1.0,27502,APEX,5.22,NaN
286829,0.0,1364500.0,2022-07,560914.0,1.88,4347,ALL,3.5,1.0,27502,APEX,5.41,3.0
286830,0.0,220000.0,2022-06,93432.0,1.00,896,ALL,1.0,1.0,27529,GARNER,5.52,3.4
286831,0.0,170000.0,2021-05,119201.0,1.00,896,ALL,1.0,1.0,27529,GARNER,2.96,4.3


## Population growth
### 2000-2010

In [8]:
pop_growth_df_1 = pop_growth_df_1.loc[pop_growth_df_1['STNAME'] == 'North Carolina']
pop_growth_df_1 = pop_growth_df_1.loc[pop_growth_df_1['CTYNAME'] == 'Wake County']
pop_growth_df_1.drop(columns=['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME'], inplace=True)

In [9]:
pop_growth_df_1['rate_2001'] = (pop_growth_df_1['POPESTIMATE2001'] - pop_growth_df_1['POPESTIMATE2000']) / pop_growth_df_1['POPESTIMATE2000']
pop_growth_df_1['rate_2002'] = (pop_growth_df_1['POPESTIMATE2002'] - pop_growth_df_1['POPESTIMATE2001']) / pop_growth_df_1['POPESTIMATE2001']
pop_growth_df_1['rate_2003'] = (pop_growth_df_1['POPESTIMATE2003'] - pop_growth_df_1['POPESTIMATE2002']) / pop_growth_df_1['POPESTIMATE2002']
pop_growth_df_1['rate_2004'] = (pop_growth_df_1['POPESTIMATE2004'] - pop_growth_df_1['POPESTIMATE2003']) / pop_growth_df_1['POPESTIMATE2003']
pop_growth_df_1['rate_2005'] = (pop_growth_df_1['POPESTIMATE2005'] - pop_growth_df_1['POPESTIMATE2004']) / pop_growth_df_1['POPESTIMATE2004']
pop_growth_df_1['rate_2006'] = (pop_growth_df_1['POPESTIMATE2006'] - pop_growth_df_1['POPESTIMATE2005']) / pop_growth_df_1['POPESTIMATE2005']
pop_growth_df_1['rate_2007'] = (pop_growth_df_1['POPESTIMATE2007'] - pop_growth_df_1['POPESTIMATE2006']) / pop_growth_df_1['POPESTIMATE2006']
pop_growth_df_1['rate_2008'] = (pop_growth_df_1['POPESTIMATE2008'] - pop_growth_df_1['POPESTIMATE2007']) / pop_growth_df_1['POPESTIMATE2007']
pop_growth_df_1['rate_2009'] = (pop_growth_df_1['POPESTIMATE2009'] - pop_growth_df_1['POPESTIMATE2008']) / pop_growth_df_1['POPESTIMATE2008']
pop_growth_df_1['rate_2010'] = (pop_growth_df_1['POPESTIMATE2010'] - pop_growth_df_1['POPESTIMATE2009']) / pop_growth_df_1['POPESTIMATE2009']

In [10]:
pop_growth_df_1.drop(columns=['ESTIMATESBASE2000', 'CENSUS2010POP', 'POPESTIMATE2000', 'POPESTIMATE2001', 'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004', 'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009', 'POPESTIMATE2010'], inplace=True)
pop_growth_df_1.set_index('CTYNAME', inplace=True)
pop_growth_df_1.head()

,rate_2001,rate_2002,rate_2003,rate_2004,rate_2005,rate_2006,rate_2007,rate_2008,rate_2009,rate_2010
CTYNAME,,,,,,,,,,
Wake County,0.037191,0.0298,0.028614,0.029594,0.041478,0.050922,0.047917,0.042459,0.031678,0.02642


### 2011-2020

In [11]:
pop_growth_df_2 = pop_growth_df_2.loc[pop_growth_df_2['STNAME'] == 'North Carolina']
pop_growth_df_2 = pop_growth_df_2.loc[pop_growth_df_2['CTYNAME'] == 'Wake County']
pop_growth_df_2.drop(columns=['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME'], inplace=True)

In [12]:
pop_growth_df_2['rate_2011'] = (pop_growth_df_2['POPESTIMATE2011'] - pop_growth_df_2['POPESTIMATE2010']) / pop_growth_df_2['POPESTIMATE2010']
pop_growth_df_2['rate_2012'] = (pop_growth_df_2['POPESTIMATE2012'] - pop_growth_df_2['POPESTIMATE2011']) / pop_growth_df_2['POPESTIMATE2011']
pop_growth_df_2['rate_2013'] = (pop_growth_df_2['POPESTIMATE2013'] - pop_growth_df_2['POPESTIMATE2012']) / pop_growth_df_2['POPESTIMATE2012']
pop_growth_df_2['rate_2014'] = (pop_growth_df_2['POPESTIMATE2014'] - pop_growth_df_2['POPESTIMATE2013']) / pop_growth_df_2['POPESTIMATE2013']
pop_growth_df_2['rate_2015'] = (pop_growth_df_2['POPESTIMATE2015'] - pop_growth_df_2['POPESTIMATE2014']) / pop_growth_df_2['POPESTIMATE2014']
pop_growth_df_2['rate_2016'] = (pop_growth_df_2['POPESTIMATE2016'] - pop_growth_df_2['POPESTIMATE2015']) / pop_growth_df_2['POPESTIMATE2015']
pop_growth_df_2['rate_2017'] = (pop_growth_df_2['POPESTIMATE2017'] - pop_growth_df_2['POPESTIMATE2016']) / pop_growth_df_2['POPESTIMATE2016']
pop_growth_df_2['rate_2018'] = (pop_growth_df_2['POPESTIMATE2018'] - pop_growth_df_2['POPESTIMATE2017']) / pop_growth_df_2['POPESTIMATE2017']
pop_growth_df_2['rate_2019'] = (pop_growth_df_2['POPESTIMATE2019'] - pop_growth_df_2['POPESTIMATE2018']) / pop_growth_df_2['POPESTIMATE2018']
pop_growth_df_2['rate_2020'] = (pop_growth_df_2['POPESTIMATE2020'] - pop_growth_df_2['POPESTIMATE2019']) / pop_growth_df_2['POPESTIMATE2020']


In [13]:
pop_growth_df_2 = pop_growth_df_2[['CTYNAME', 'rate_2011', 'rate_2012', 'rate_2013', 'rate_2014', 'rate_2015', 'rate_2016', 'rate_2017', 'rate_2018', 'rate_2019', 'rate_2020']]
pop_growth_df_2.set_index('CTYNAME', inplace=True)
pop_growth_df_2.head()

,rate_2011,rate_2012,rate_2013,rate_2014,rate_2015,rate_2016,rate_2017,rate_2018,rate_2019,rate_2020
CTYNAME,,,,,,,,,,
Wake County,0.024292,0.025149,0.023357,0.024333,0.024694,0.025479,0.022432,0.018405,0.018855,0.017201


### 2021

In [14]:
pop_growth_df_3 = pop_growth_df_3.loc[pop_growth_df_3['STNAME'] == 'North Carolina']
pop_growth_df_3 = pop_growth_df_3.loc[pop_growth_df_3['CTYNAME'] == 'Wake County']
pop_growth_df_3.drop(columns=['SUMLEV', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'STNAME'], inplace=True)

In [15]:
pop_growth_df_3['rate_2021'] = (pop_growth_df_3['POPESTIMATE2021'] - pop_growth_df_3['POPESTIMATE2020']) / pop_growth_df_3['POPESTIMATE2020']
pop_growth_df_3 = pop_growth_df_3[['CTYNAME', 'rate_2021']].copy()
pop_growth_df_3.set_index('CTYNAME', inplace=True)

pop_growth_df_3.head()


,rate_2021
CTYNAME,
Wake County,0.014689


#### Combine all population growth datasets together

In [16]:
pop_growth_df = pd.concat([pop_growth_df_1, pop_growth_df_2, pop_growth_df_3], axis=1)
pop_growth_df.head()

,rate_2001,rate_2002,rate_2003,rate_2004,rate_2005,rate_2006,rate_2007,rate_2008,rate_2009,rate_2010,...,rate_2012,rate_2013,rate_2014,rate_2015,rate_2016,rate_2017,rate_2018,rate_2019,rate_2020,rate_2021
CTYNAME,,,,,,,,,,,,,,,,,,,,,
Wake County,0.037191,0.0298,0.028614,0.029594,0.041478,0.050922,0.047917,0.042459,0.031678,0.02642,...,0.025149,0.023357,0.024333,0.024694,0.025479,0.022432,0.018405,0.018855,0.017201,0.014689


In [17]:
final_pop_growth = pop_growth_df.transpose()
final_pop_growth.reset_index(inplace=True)
final_pop_growth.rename(columns={'index':'year', 'Wake County':'pop_growth_rate'}, inplace=True)
final_pop_growth['year'] = final_pop_growth['year'].str.split('_').str[1]
final_pop_growth['year'] = pd.to_datetime(final_pop_growth['year'], format='%Y')
final_pop_growth['year'] = final_pop_growth['year'].dt.strftime('%Y')

final_pop_growth.rename_axis(None, axis=1, inplace=True)
final_pop_growth.head()

,year,pop_growth_rate
0,2001,0.037191
1,2002,0.029800
2,2003,0.028614
3,2004,0.029594
4,2005,0.041478


In [18]:
updated_house_df['year'] = pd.to_datetime(updated_house_df['deed_date']).dt.year
updated_house_df['year'] = updated_house_df['year'].astype(str)
updated_house_df = updated_house_df.merge(final_pop_growth, left_on='year', right_on='year', how='left')
updated_house_df.drop(columns=['year'], inplace=True)
updated_house_df

,land_sale_price,total_sale_price,deed_date,assessed_building_value,story_height,heated_area,utilities,bath,type_and_use,physical_zip_code,physical_city,mort_rate,unemploy_rate,pop_growth_rate
0,0.0,37500.0,2004-09,118723.0,1.00,2261,WSE,2.0,1.0,27591,WENDELL,5.75,4.2,0.029594
1,0.0,380000.0,2015-08,161077.0,1.00,1789,ALL,2.0,1.0,27607,RALEIGH,3.91,4.9,0.024694
2,0.0,337500.0,2012-12,273621.0,1.00,2463,ALL,2.0,1.0,27615,RALEIGH,3.35,7.0,0.025149
3,0.0,319000.0,2010-06,503301.0,1.50,4650,ALL,3.5,1.0,27608,RALEIGH,4.74,8.4,0.026420
4,0.0,425000.0,2008-04,203178.0,2.00,1890,ALL,2.0,1.0,27604,RALEIGH,5.92,3.7,0.042459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286828,0.0,1194000.0,2022-08,547787.0,1.88,4520,ALL,3.5,1.0,27502,APEX,5.22,NaN,NaN
286829,0.0,1364500.0,2022-07,560914.0,1.88,4347,ALL,3.5,1.0,27502,APEX,5.41,3.0,NaN
286830,0.0,220000.0,2022-06,93432.0,1.00,896,ALL,1.0,1.0,27529,GARNER,5.52,3.4,NaN
286831,0.0,170000.0,2021-05,119201.0,1.00,896,ALL,1.0,1.0,27529,GARNER,2.96,4.3,0.014689


## COVID data

In [19]:
nc_covid = covid_df.loc[covid_df['State'] == 'NC']
# nc_covid.head()
wake_covid = nc_covid.loc[nc_covid['County Name'] == 'Wake County ']
wake_covid.drop(columns=['countyFIPS', 'StateFIPS', 'State', 'County Name'], inplace=True)
wake_covid.reset_index(drop=True, inplace=True)
wake_covid.head()

c:\Users\aliso\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,1/22/2020,1/23/2020,1/24/2020,1/25/2020,1/26/2020,1/27/2020,1/28/2020,1/29/2020,1/30/2020,1/31/2020,...,9/6/2022,9/7/2022,9/8/2022,9/9/2022,9/10/2022,9/11/2022,9/12/2022,9/13/2022,9/14/2022,9/15/2022
0,0,0,0,0,0,0,0,0,0,0,...,359237,361248,361248,361248,361248,361248,361248,361248,363398,363398


In [20]:
final_wake_covid = wake_covid.transpose()
final_wake_covid.reset_index(inplace=True)
final_wake_covid.rename(columns={'index':'date', 0:'covid_cases'}, inplace=True)
final_wake_covid['date'] = pd.to_datetime(final_wake_covid['date'])
final_wake_covid['date'] = final_wake_covid['date'].dt.strftime('%Y-%m')
final_wake_covid


,date,covid_cases
0,2020-01,0
1,2020-01,0
2,2020-01,0
3,2020-01,0
4,2020-01,0
...,...,...
962,2022-09,361248
963,2022-09,361248
964,2022-09,361248
965,2022-09,363398


In [21]:
updated_house_df = updated_house_df.merge(final_wake_covid, left_on='deed_date', right_on='date', how='left')
updated_house_df.drop(columns=['date'], inplace=True)

updated_house_df['is_covid'] = 0
updated_house_df['deed_date'] = pd.to_datetime(updated_house_df['deed_date'])
# updated_house_df.info()
covid_house_df = updated_house_df.loc[updated_house_df['deed_date'] >= '2020-03-01']
covid_house_df['is_covid'] = 1
uncovid_house_df = updated_house_df.loc[updated_house_df['deed_date'] < '2020-03-01']
uncovid_house_df['is_covid'] = 0
updated_house_df = pd.concat([covid_house_df, uncovid_house_df], axis=0)
updated_house_df.reset_index(drop=True, inplace=True)
updated_house_df


C:\Users\aliso\AppData\Local\Temp/ipykernel_25860/2332757497.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_house_df['is_covid'] = 1
C:\Users\aliso\AppData\Local\Temp/ipykernel_25860/2332757497.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uncovid_house_df['is_covid'] = 0


,land_sale_price,total_sale_price,deed_date,assessed_building_value,story_height,heated_area,utilities,bath,type_and_use,physical_zip_code,physical_city,mort_rate,unemploy_rate,pop_growth_rate,covid_cases,is_covid
0,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,74599.0,1
1,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,74696.0,1
2,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,74957.0,1
3,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,75183.0,1
4,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,75405.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744451,0.0,280000.0,2020-02-01,224222.0,2.0,2446,ALL,2.5,1.0,27587,WAKE FOREST,3.47,3.2,0.017201,0.0,0
2744452,0.0,280000.0,2020-02-01,224222.0,2.0,2446,ALL,2.5,1.0,27587,WAKE FOREST,3.47,3.2,0.017201,0.0,0
2744453,0.0,280000.0,2020-02-01,224222.0,2.0,2446,ALL,2.5,1.0,27587,WAKE FOREST,3.47,3.2,0.017201,0.0,0
2744454,0.0,280000.0,2020-02-01,224222.0,2.0,2446,ALL,2.5,1.0,27587,WAKE FOREST,3.47,3.2,0.017201,0.0,0


## Raw materials data
### Lumber

In [22]:
lumber_df.rename(columns={'DATE':'date', 'WPU081':'lumber_price'}, inplace=True)
lumber_df['date'] = pd.to_datetime(lumber_df['date'])
lumber_df['date'] = lumber_df['date'].dt.strftime('%Y-%m')
lumber_df = lumber_df[lumber_df['date'] >= '2000-01']
lumber_df.reset_index(drop=True, inplace=True)
lumber_df.head()

,date,lumber_price
0,2000-01,189.1
1,2000-02,189.2
2,2000-03,188.5
3,2000-04,186.3
4,2000-05,180.3


### Metals

In [23]:
metals_df.rename(columns={'DATE':'date', 'WPU101':'metals_price'}, inplace=True)
metals_df['date'] = pd.to_datetime(metals_df['date'])
metals_df['date'] = metals_df['date'].dt.strftime('%Y-%m')
metals_df = metals_df[metals_df['date'] >= '2000-01']
metals_df.reset_index(drop=True, inplace=True)
metals_df.head()

,date,metals_price
0,2000-01,118.3
1,2000-02,118.2
2,2000-03,118.4
3,2000-04,119.0
4,2000-05,118.4


### Build materials

In [24]:
build_df.rename(columns={'DATE':'date', 'PCU44414441':'build_price'}, inplace=True)
build_df['date'] = pd.to_datetime(build_df['date'])
build_df['date'] = build_df['date'].dt.strftime('%Y-%m')
build_df = build_df[build_df['date'] >= '2000-01']
build_df.reset_index(drop=True, inplace=True)
build_df.head()

,date,build_price
0,2003-12,100.0
1,2004-01,102.5
2,2004-02,104.1
3,2004-03,105.1
4,2004-04,108.6


### Combine all together

In [25]:
lumber_metals_df = lumber_df.merge(metals_df, left_on='date', right_on='date', how='left')

final_material_df = lumber_metals_df.merge(build_df, left_on='date', right_on='date', how='left')
final_material_df.fillna(0, inplace=True)
final_material_df.head()

,date,lumber_price,metals_price,build_price
0,2000-01,189.1,118.3,0.0
1,2000-02,189.2,118.2,0.0
2,2000-03,188.5,118.4,0.0
3,2000-04,186.3,119.0,0.0
4,2000-05,180.3,118.4,0.0


### Combine raw material data to house data

In [26]:
final_material_df['date'] = pd.to_datetime(final_material_df['date'])
updated_house_df = updated_house_df.merge(final_material_df, left_on='deed_date', right_on='date', how='left')
updated_house_df.drop(columns=['date'], inplace=True)
updated_house_df.head()

,land_sale_price,total_sale_price,deed_date,assessed_building_value,story_height,heated_area,utilities,bath,type_and_use,physical_zip_code,physical_city,mort_rate,unemploy_rate,pop_growth_rate,covid_cases,is_covid,lumber_price,metals_price,build_price
0,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,74599.0,1,365.8,292.2,198.9
1,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,74696.0,1,365.8,292.2,198.9
2,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,74957.0,1,365.8,292.2,198.9
3,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,75183.0,1,365.8,292.2,198.9
4,0.0,395000.0,2021-03-01,101436.0,1.0,1212,ALL,2.0,1.0,27608,RALEIGH,3.08,4.4,0.014689,75405.0,1,365.8,292.2,198.9


# Save as a CSV file

In [27]:
# updated_house_df.to_csv('assets/updated_house_df.csv', index=False)